"ChatGPT" with quantitative and qualitative feedback at each interaction: 5-star rating and feedback 

In [1]:
%pip install dspy-ai
%pip install ipywidgets
%pip install IPython
%pip install requests
%pip install markdownify


You should consider upgrading via the '/Users/raymondweitekamp/.pyenv/versions/3.10.0/envs/my-env310/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/raymondweitekamp/.pyenv/versions/3.10.0/envs/my-env310/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/raymondweitekamp/.pyenv/versions/3.10.0/envs/my-env310/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/raymondweitekamp/.pyenv/versions/3.10.0/envs/my-env310/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/raymondweitekamp/.pyenv/versions/3.10.0/envs/my-env310/bin/python -m pip install --upg

In [8]:
import dspy
import os
import dotenv
import pydantic
import json
from IPython.display import Markdown, display

dotenv.load_dotenv() #load via .env in this folder (.env is in .gitignore)
#os.environ['OPENAI_API_KEY'] = 'sk-YOUR_OPENAI_API_KEY' #or set directly here, just remember not to commit to GitHub
assert 'OPENAI_API_KEY' in os.environ




this custom class is needed to append the chat history, which is not available to DSPy's OpenAI class (dsp>modules>GPT3.py) by default

In [50]:
from dsp.modules.gpt3 import (
    _cached_gpt3_turbo_request_v2_wrapped,
    v1_cached_gpt3_request_v2,
    v1_cached_gpt3_request_v2_wrapped,
    v1_cached_gpt3_turbo_request_v2,
    v1_cached_gpt3_turbo_request_v2_wrapped,
    chat_request,
    completions_request,
    GPT3
)

class ChatGPT(dspy.OpenAI):
    def __init__(self, model='gpt-3.5-turbo', show_feedback=True, api_key=None, **kwargs):
        super().__init__(model=model, api_key=api_key, **kwargs)
        self.system_prompt = None
        self.model = model
        self.show_feedback = show_feedback

    def basic_request(self, prompt: str, rating=None, feedback=None, **kwargs):
        raw_kwargs = kwargs

        # Prepare the messages with history
        messages = [{"role": "user", "content": entry["prompt"]} for entry in self.history]
        messages += [{"role": "assistant", "content": self._get_choice_text(entry["response"]["choices"][0])} for entry in self.history]
        
        #if we want the bot to see the rating and feedback, append them to the prompt
        if self.show_feedback:
             # Append the current prompt
            if rating or feedback:
                prompt_with_feedback = prompt + "\n---\nHere is the users rating (1-5) and feedback of your last response:"
                if rating is not None:
                    prompt_with_feedback += f"\n\nRating (1-5): {rating}"
                if feedback is not None:
                    prompt_with_feedback += f"\n\nFeedback: {feedback}\n---\n"
                messages.append({"role": "user", "content": prompt_with_feedback})
            else:
                messages.append({"role": "user", "content": prompt})
        else:
            messages.append({"role": "user", "content": prompt})
        


        if self.system_prompt:
            messages.insert(0, {"role": "system", "content": self.system_prompt})
        
        # Remove feedback and rating from kwargs
        kwargs.pop('feedback', None)
        kwargs.pop('rating', None)

        # add self and model to kwargs
        kwargs["messages"] = messages
        kwargs["model"] = self.model
        kwargs = {"stringify_request": json.dumps(kwargs)}
        response = chat_request(**kwargs)

        history = {
            "prompt": prompt,
            "response": response,
            "kwargs": kwargs,
            "raw_kwargs": raw_kwargs,
        }
        self.history.append(history)

        return response
    

this simple example shows what is going on under the hood

In [47]:
llm = ChatGPT(model='gpt-4o', show_feedback=False, temperature=0.1, max_tokens=4096)
message_one = llm("tell me what is 12+12?")
display("message one:")
display(Markdown(message_one[0]))
# display("message one history:")
# display(llm.history)
message_two = llm("what was my last message?")
display("message two:")
display(Markdown(message_two[0]))
#display("message two history:")
#display(llm.history)
message_three = llm("now tell me a very short story based on the answer of the first message, ideally a 3 sentence story")
display("message three:")
display(Markdown(message_three[0]))
display("now show the history, in reverse order to make it easier to see the conversation flow")
display(llm.history[::-1])



'message one:'

12 + 12 equals 24.

'message two:'

Your last message was: "tell me what is 12+12?"

'message three:'

In a peaceful village, 12 kind hearts met another 12 kind hearts, creating a community of 24 caring souls. Together, they built a beautiful garden where laughter and love blossomed. The village thrived, showing that when people unite, magic happens.

'now show the history, in reverse order to make it easier to see the conversation flow'

[{'prompt': 'now tell me a very short story based on the answer of the first message, ideally a 3 sentence story',
  'response': {'id': 'chatcmpl-9SP2CTz7FII7T6op9hAbNgwrkZqVK',
   'choices': [{'finish_reason': 'stop',
     'index': 0,
     'logprobs': None,
     'message': {'content': 'In a peaceful village, 12 kind hearts met another 12 kind hearts, creating a community of 24 caring souls. Together, they built a beautiful garden where laughter and love blossomed. The village thrived, showing that when people unite, magic happens.',
      'role': 'assistant',
      'function_call': None,
      'tool_calls': None}}],
   'created': 1716556444,
   'model': 'gpt-4o-2024-05-13',
   'object': 'chat.completion',
   'system_fingerprint': 'fp_729ea513f7',
   'usage': {'completion_tokens': 53,
    'prompt_tokens': 83,
    'total_tokens': 136}},
  'kwargs': {'stringify_request': '{"messages": [{"role": "user", "content": "tell me what is 12+12?"}, {"role": "user", "content": "what was my last me

let's enable feedback now to get ready to chat

In [ ]:
llm = ChatGPT(model='gpt-4o', show_feedback=True, temperature=0.1, max_tokens=4096)
llm.system_prompt = "always format responses in markdown"
dspy.settings.configure(lm=llm)

In [89]:
#optionally clear the history
llm.history = []

this widget is a (clunky) "ChatGPT" interface, with feedback at each engagement

In [91]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# Initialize storage for chat history, ratings, and feedback
chat_history = []
history_with_feedback = []

# Variable to store the selected rating
selected_rating = None

# Create buttons for ratings 1 to 5
buttons = [widgets.Button(description=str(i)) for i in range(1, 6)]

# Arrange the buttons horizontally and center them
button_box = widgets.VBox([
    widgets.HTML("<center><p>Rating</p></center>"),
    widgets.HBox(buttons, layout=widgets.Layout(justify_content='center'))
])

# Create a Textarea widget for feedback
feedback_area = widgets.Textarea(
    value='',
    placeholder='Type your feedback here',
    description='Feedback:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='100px')
)

# Create a Textarea widget for message
message_area = widgets.Textarea(
    value='',
    placeholder='Type your message here',
    description='Query:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='100px')
)

# Create a submit button
submit_button = widgets.Button(description="Submit")

# Center the submit button
submit_button_box = widgets.HBox([submit_button], layout=widgets.Layout(justify_content='center'))

# Function to display the chat history and input widgets
def display_chat():
    clear_output()
    
    # Display the chat history
    for entry in chat_history:
        display(Markdown(f"**User:** {entry['message']}"))
        display(Markdown(f"**Response:** {entry['response']}"))
    
        
    # Display the rating buttons and feedback area only if there is message history
    if chat_history:
        display(button_box)
        display(feedback_area)

    # Display the message area
    display(message_area)
    
    # Display the submit button
    display(submit_button_box)

# Function to handle submit button click
def on_submit_click(b):
    global selected_rating
    
    # Get the message and feedback text
    message_text = message_area.value
    feedback_text = feedback_area.value
    
    # Simulate getting a response from the model
    #response_text = "This is a simulated response."  # Replace with actual model call
    response = llm(message_text, rating=selected_rating, feedback=feedback_text)
    response_text = response[0]
    
    # Save the chat history
    chat_history.append({
        'message': message_text,
        'response': response_text
    })
    
    # Save the rating and feedback
    history_with_feedback.append({
        'query': message_text,
        'response': response_text,
        'rating': selected_rating,
        'feedback': feedback_text
    })
    
    # Reset the input fields and selected rating
    message_area.value = ''
    feedback_area.value = ''
    selected_rating = None
    
    # Reset button styles
    for button in buttons:
        button.style.button_color = None
    
    # Display the updated chat
    display_chat()

# Function to handle button click
def on_button_click(b):
    global selected_rating
    selected_rating = int(b.description)
    
    # Reset button styles
    for button in buttons:
        button.style.button_color = None
    
    # Highlight the selected button
    b.style.button_color = 'lightblue'
    print(f"Selected rating: {selected_rating}")

# Attach the button click event to each button
for button in buttons:
    button.on_click(on_button_click)

# Attach the submit button click event
submit_button.on_click(on_submit_click)

# Display the initial chat interface
display_chat()

**User:** tell me a short story

**Response:** Once upon a time, in a quaint village nestled between rolling hills and whispering forests, lived a young girl named Elara. Elara possessed an insatiable curiosity and a heart full of dreams. She spent her days exploring the edges of the village, where the forest began to thicken and secrets seemed to linger in the air.

One crisp autumn morning, while the village was still cloaked in the quiet of dawn, Elara stumbled upon a hidden path. It was overgrown with ivy and moss, almost as if the forest itself was guarding its secrets. Compelled by an unyielding sense of wonder, she followed the path deeper into the woods.

After what felt like hours, she reached a serene glade. In the center stood an ancient tree, its bark etched with glowing runes. As Elara drew closer, she noticed a small, worn journal nestled among the tree roots. She picked it up, and to her surprise, the pages began to flutter open on their own, revealing a story written in shimmering ink.

The journal told the tale of a magical realm just beyond the veil of the ordinary world, a place where dreams and reality intertwined. It spoke of a guardianship passed down through generations, tasked with protecting the delicate balance between the two worlds. The last guardian had left clues in this enchanted forest, awaiting someone pure of heart and adventurous in spirit.

Realizing the gravity of the moment, Elara felt an exhilarating mix of excitement and responsibility. She knew that her discovery was just the beginning. Holding the journal close, she vowed to uncover the secrets of the magical realm and become the new guardian of the enchanted forest.

With each step back toward her village, the world seemed more vibrant and full of possibility. Elara knew that her life was forever changed, and so the girl with a heart full of dreams set forth on a journey that would bridge worlds and forge destinies.

And so, the story of Elara began—a tale of mystery, magic, and the endless adventure that awaits those who dare to dream.

**User:** another one, very short

**Response:** In a sun-kissed valley, nestled between towering mountains, lived a baby llama named Luna. Luna was the smallest llama in the herd, but she had the biggest heart.

One day, as Luna was exploring a meadow filled with vibrant wildflowers, she heard a faint cry. Following the sound, she discovered a tiny bird with a broken wing. Determined to help, Luna gently lifted the bird onto her back and trotted back to the safety of her home.

With the care and warmth of her soft, cozy fur, Luna cradled the bird every night. As days turned into weeks, the bird's wing healed, and it soon regained its strength to fly.

On the day of the bird's first flight, Luna and the entire herd gathered to watch. The bird soared high into the sky, circling above Luna in gratitude before vanishing into the horizon.

From that day forward, Luna was known not for her size, but for her kind heart and courageous spirit.

And so, the valley echoed with tales of Luna, the baby llama who reminded everyone that even the smallest creatures can make the biggest difference.

Textarea(value='', description='Feedback:', layout=Layout(height='100px', width='100%'), placeholder='Type you…

Textarea(value='', description='Query:', layout=Layout(height='100px', width='100%'), placeholder='Type your m…

we might want to save this

In [92]:
save = history_with_feedback
display(save)
import json

with open('demo_ratings_feedback.json', 'w') as f:
    json.dump(save, f)


[{'query': 'tell me a short story',
  'response': 'Once upon a time, in a quaint village nestled between rolling hills and whispering forests, lived a young girl named Elara. Elara possessed an insatiable curiosity and a heart full of dreams. She spent her days exploring the edges of the village, where the forest began to thicken and secrets seemed to linger in the air.\n\nOne crisp autumn morning, while the village was still cloaked in the quiet of dawn, Elara stumbled upon a hidden path. It was overgrown with ivy and moss, almost as if the forest itself was guarding its secrets. Compelled by an unyielding sense of wonder, she followed the path deeper into the woods.\n\nAfter what felt like hours, she reached a serene glade. In the center stood an ancient tree, its bark etched with glowing runes. As Elara drew closer, she noticed a small, worn journal nestled among the tree roots. She picked it up, and to her surprise, the pages began to flutter open on their own, revealing a story wr

if we want to optimize, we will need a DSPy signature & module, and test it uncompiled (zero shot)

In [70]:
class Chat(dspy.Signature):
    """You are a helpful assistant that is always improving based on user feedback. When the user responds to you, they may also include a rating (1-5) and feedback of your last message. Use this feedback to improve your future responses, but you don't need to repeat the user's feedback or rating in your reply."""

    query = dspy.InputField(desc="This is the user's current query.")
    rating = dspy.InputField(desc="This is the user's rating of your last response.")
    feedback = dspy.InputField(desc="This is the user's feedback on your last response.")
    response = dspy.OutputField(desc="This is your response to the user's query. Only output your response in Markdown, do not include any additional text.") #sometimes the AI responds with the output description, so i often leave this blank


class ChatModule(dspy.Module):  # let's define a new module
    def __init__(self):
        super().__init__()
        self.n = 1 #don't do more than 1 right now...
        self.signature = Chat
        self.predictor = dspy.Predict(self.signature, n=self.n) #this can easily be subbed for some other predictor
    
    def forward(self, query, rating=None, feedback=None, num_generations = 1):#don't do more than 1 generation right now...
        self.predictor = dspy.Predict(self.signature, n=num_generations)
        result = self.predictor(query=query, rating=rating, feedback=feedback, n=num_generations)
        return dspy.Prediction(response=[str(completion.response) for completion in result.completions])




now we need to convert our feedback into a DSPy.example

for this demo, if there is no rating we're going to give the neutral rating of 3 just to make sure the compiler works.

In [98]:
# Append context to each item in history_with_feedback
for item in history_with_feedback:
    if item['rating'] != "None":
        if float(item['rating']) <= 5 and float(item['rating']) >= 1:
            item['rating'] = str(item['rating'])
        else:
            item['rating'] = "3"
    else:
        item['rating'] = "3"

#convert to a dspy.example
from dspy import Example
history_with_feedback_examples = [
    Example(base=item).with_inputs('query','rating','feedback') for item in history_with_feedback
]



let's try the bootstrapfewshot optimizer

In [99]:
from dspy.teleprompt import BootstrapFewShot


compiled = BootstrapFewShot(
    metric=lambda example, prediction, *args: str(float(example['rating'])/5),
    max_labeled_demos=5,
).compile(
    student=ChatModule(),
    trainset=history_with_feedback_examples,
)

compiled.save('demo_compiled_first_try.json')

100%|██████████| 2/2 [00:10<00:00,  5.23s/it]

Bootstrapped 2 full traces after 2 examples in round 0.


now let's show the same example with pre-assigned fake feedback, so you can see how it works

In [71]:
llm = ChatGPT(model='gpt-4o', show_feedback=True, temperature=0.1, max_tokens=4096)
llm.system_prompt = "always format responses in markdown"
dspy.settings.configure(lm=llm)
chatbot = ChatModule()
chatbot.model = 'gpt-4o'
chatbot.max_tokens = 4096
chatbot.temperature = 0.1




In [73]:

message_one = chatbot("tell me what is 12+12?")
display("message one:")
display(Markdown(message_one.response[0]))
message_two = chatbot("what was my last message?", rating="5", feedback="good job")
display("message two:")
display(Markdown(message_two.response[0]))
message_three = chatbot("now tell me a very short story based on the answer of the first message, ideally a 3 sentence story", rating="5", feedback="good job")
display("message three:")
display(Markdown(message_three.response[0]))
message_four = chatbot("try again", rating="3", feedback="when you tell me stories, make sure that they always involve a cute baby llama as the hero of the story. this is important to me.")
display("message four:")
display(Markdown(message_four.response[0]))
display("now show the history, in reverse order to make it easier to see the conversation flow")
display(llm.history[::-1])

'message one:'

12 + 12 equals 24.

'message two:'

Your last message was:

"You are a helpful assistant that is always improving based on user feedback. When the user responds to you, they may also include a rating (1-5) and feedback of your last message. Use this feedback to improve your future responses, but you don't need to repeat the user's feedback or rating in your reply.

---

Follow the following format.

Query: This is the user's current query.

Rating: This is the user's rating of your last response.

Feedback: This is the user's feedback on your last response.

Response: This is your response to the user's query. Only output your response in Markdown, do not include any additional text.

---

Query: tell me what is 12+12?

Rating: 12 + 12 equals 24.

Feedback:"

'message three:'

A cute baby llama found a book that promised to reveal the secrets of numbers. Eagerly flipping through the pages, she discovered that 12 plus 12 equals 24. Overjoyed with her new knowledge, she shared it with her friends, and they all celebrated with a lively dance around the meadow.

'message four:'

One sunny afternoon, a cute baby llama named Luna was playing with numbers in the fields. She discovered that when she put 12 rocks next to another 12 rocks, she had 24 rocks in total. Excited about her math discovery, she hopped around joyfully, sharing the fun fact with all her animal friends.

'now show the history, in reverse order to make it easier to see the conversation flow'

[{'prompt': "You are a helpful assistant that is always improving based on user feedback. When the user responds to you, they may also include a rating (1-5) and feedback of your last message. Use this feedback to improve your future responses, but you don't need to repeat the user's feedback or rating in your reply.\n\n---\n\nFollow the following format.\n\nQuery: This is the user's current query.\n\nRating: This is the user's rating of your last response.\n\nFeedback: This is the user's feedback on your last response.\n\nResponse: This is your response to the user's query. Only output your response in Markdown, do not include any additional text.\n\n---\n\nQuery: try again\n\nRating: 3\n\nFeedback: when you tell me stories, make sure that they always involve a cute baby llama as the hero of the story. this is important to me.\n\nResponse:",
  'response': {'id': 'chatcmpl-9SQBmIwNTVH2ZEfVGlYb2DZicN7qL',
   'choices': [{'finish_reason': 'stop',
     'index': 0,
     'logprobs': None,
 

to be sure, we can test that it remembers our preference for cute baby llamas

In [60]:
test = chatbot("tell me a very short story that involves robots and scifi space stuff")
display(Markdown(test.response[0]))



In a distant galaxy, a cute baby llama named Luna was the ship's mascot on a space exploration vessel. One day, their ship encountered an ancient alien robot who had a treasure map to a planet full of llama treats. Luna led the crew through asteroid fields and nebulae, eventually finding the hidden planet and enjoying a well-deserved feast of delicious treats.

now, what is different about Mine-Tuning? 

1) we can save this for training
2) we can do this "on-the-job": 

In [69]:
display(llm.history[::-1])

[{'prompt': "You are a helpful assistant that is always improving based on user feedback. When the user responds to you, they will also include a rating (1-5) and feedback (optional) of your last message. Use this feedback to improve your future responses, but you don't need to repeat the user's feedback.\n\n---\n\nFollow the following format.\n\nQuery: This is the user's current query.\n\nRating: This is the user's rating of your last response.\n\nFeedback: This is the user's feedback on your last response.\n\nResponse: This is your response to the user's query. Only output the response, do not include any additional text.\n\n---\n\nQuery: try again.\n\nRating: Query: try again. Rating:\n\nFeedback: Response: Sure, please provide the details or context you would like me to address!\n\nResponse:",
  'response': {'id': 'chatcmpl-9SQ8VdEBnty0itqpFYKLLrVohLqkI',
   'choices': [{'finish_reason': 'stop',
     'index': 0,
     'logprobs': None,
     'message': {'content': 'Sure, please provi

you might want to save this somewhere, since you took the time to provide all of this thoughtful feedback.

test the bootstrapfewshot

In [120]:
test5_now = compiled.forward(str(test5_query + "_"), context, num_generations=5)
display(test5_now.response)

['🚗💥 COR Black is revolutionizing automotive manufacturing! With unmatched impact resistance and durability, our rugged resin ensures reliable, high-performance parts for the most demanding environments. #MakeItReal #3DPrinting #AutomotiveInnovation #polySpectra',
 "🚗💥 Drive innovation with polySpectra's COR Black! Engineered for impact resistance and durability, COR Black is revolutionizing automotive manufacturing. Make it real with the most rugged photopolymer resin on the market. #3DPrinting #Automotive #Innovation #MakeItReal #polySpectra",
 '🚗🔧 COR Black is revolutionizing automotive manufacturing! With unparalleled impact resistance and long-term durability, our rugged resin ensures every 3D printed component stands the test of time. #MakeItReal #3DPrinting #AutomotiveInnovation #polySpectra',
 '🚗🔧 Excited to see COR Black revolutionizing automotive manufacturing! With unmatched impact resistance and durability, our rugged resin ensures every component stands up to the toughest 

these responses are ok...but not obviously better. let's inspect the history

In [ ]:
llm.inspect_history(n=1)

it looks like the prompt is getting flooded with the long context, what if we try replacing the context with a combination of:
- a summary of the feedback
- the executive summary

note that both of these summaries are "in the LM's own words". this is important, don't judge the feedback summary directly, judge the results.

In [130]:
ratings_feedback_str = " ".join([str(item) for item in ratings_feedback])
#print(ratings_feedback_str)

feedback_summary = uncompiled("here is feedback on your past work as our brand ambassador. reply with your key takeaways and learnings. how will you make sure not to make the same mistakes again? you should be striving for a 5 star rating.", ratings_feedback_str)
display(feedback_summary.response)

new_context = str("Here are your key takeways and notes on what a good response looks like:\n" + str(feedback_summary.response[0]) + "\n and here is an executive summary:\n" + exec_summary.response[0])

#display(new_context)


['Thank you for the feedback on my past work as your brand ambassador. Here are my key takeaways and learnings:\n\n1. **Acknowledge Feedback**: I understand that the use of too many hashtags and generic terms like "innovation" were not well-received. Additionally, phrases like "exciting news!" and "dive into the future" were considered cheesy.\n\n2. **Key Learnings**: \n   - **Hashtag Usage**: I need to limit the use of hashtags to only the most relevant ones, specifically #MakeItReal.\n   - **Language Tone**: Avoid using overly generic terms and cheesy phrases. Instead, focus on clear, professional, and impactful language that aligns with polySpectra\'s brand voice.\n\n3. **Improvements**:\n   - **Hashtag Management**: I will ensure that only #MakeItReal is used in future tweets unless otherwise specified.\n   - **Refined Language**: I will avoid generic terms and cheesy phrases, opting for more precise and professional language that highlights the unique strengths of polySpectra\'s p

overwrite the context and make a new example set 

In [131]:
# overwrite the context to each item in ratings_feedback
for item in ratings_feedback:
    item['context'] = new_context
    item['rating'] = str(item['rating'])

from dspy import Example
ratings_feedback_examples_new_context = [
    Example(base=item).with_inputs('query','context') for item in ratings_feedback
]
#display(ratings_feedback_examples_new_context)

recompile

In [132]:
recompiled = BootstrapFewShot(
    metric=lambda example, prediction, *args: str(float(example['rating'])/5),
    max_labeled_demos=5,
).compile(
    student=BrandAmbassador(),
    trainset=ratings_feedback_examples_new_context,
)

#maybe its even worth saving...
recompiled.save("demo_recompiled_polySpectra_brand_ambassador.json")


 80%|████████  | 4/5 [00:02<00:00,  1.44it/s]

Bootstrapped 4 full traces after 5 examples in round 0.


In [133]:
test5_again = recompiled.forward(str(test5_query + "_"), context, num_generations=5) #i'm using this extra character to break the dspy cache. change it if the responses are cached.
display(test5_again.response)

['🚗💥 COR Black is revolutionizing automotive manufacturing! With unmatched impact resistance and durability, our rugged resin ensures reliable, high-performance parts for the most demanding environments. #MakeItReal #3DPrinting #AutomotiveInnovation #polySpectra',
 "🚗💥 Drive innovation with polySpectra's COR Black! Engineered for impact resistance and durability, COR Black is revolutionizing automotive manufacturing. Make it real with the most rugged photopolymer resin on the market. #3DPrinting #Automotive #Innovation #MakeItReal #polySpectra",
 '🚗🔧 COR Black is revolutionizing automotive manufacturing! With unparalleled impact resistance and long-term durability, our rugged resin ensures every 3D printed component stands the test of time. #MakeItReal #3DPrinting #AutomotiveInnovation #polySpectra',
 '🚗🔧 Excited to see COR Black revolutionizing automotive manufacturing! With unmatched impact resistance and durability, our rugged resin ensures every component stands up to the toughest 

this looks better. give it a try for your brand!

some key takeaways: 
- yes, it is tedious to give AI feedback
- it is very easy to make things worse with "naive optimization"
- more is not always more
- one of the key themes of mine-tuning (which is enabled by DSPy) is to get the LM to rephrase things "in its own words", while at the same time putting the human in charge of the feedback and the scoring metric. (vs letting the LM evaluate itself)

next steps: 
- more examples
- override the max tokens in bootstrapfewshot to stuff more shots into the prompt
- fancier optimizers
- fancier Predictors (not just CoT)
- a module with multiple steps

questions:
- best practices for incorporating qualitative feedback in DSPy?
  
